In [1]:
import openmdao.api as om
from src.v1.symbolic import Var, Par
from src.v1.execution import Evaluable
from src.v1.mdaobuild import Expcomp

In [2]:
p = Par('p', 0.1)
b,c,d = Var('b'), Var('c'), Var('d')
eqs = {
    2: (c, p*d+20+0.5*b), # c is an output that depends on d
    3: (d, c - 10),       # d is an output that depends on c
}

In [4]:
prob = om.Problem()
model = prob.model

bal = om.BalanceComp()
bal.add_balance('c', val=1.0, rhs_name=f'rhs_c')
bal.add_balance('d', val=1.0, rhs_name=f'rhs_d')
model.add_subsystem(name='balance', subsys=bal, promotes=['*'])

# Add the equation components to the main model
for eqn in [2,3]:
    (var, rhs_expr) = eqs[eqn]
    model.add_subsystem(
        f'rhs_{var}_comp',
        Expcomp(
            output_name=f'rhs_{var}',
            equation=Evaluable.fromsympy(rhs_expr-var),
            debug=False
        ),
        promotes=['*']
    )

# Solvers
model.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
model.linear_solver = om.DirectSolver()
prob.setup()

# # Set parameter value
prob.set_val('p', 0.1)

prob.run_model()
print(f"c = {prob.get_val('c')}")
print(f"d = {prob.get_val('d')}")

NL: Newton Converged in 1 iterations
c = [20.44444444]
d = [9.44444444]
